# This notebook is used to test the data loading modules (storage.py, extraction.py)

In [1]:

import h5py
import logging
import logging.config
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pathlib import Path
import string
import yaml

import sys
sys.path.append('/src/capstone-prototype/src/')
from config import settings
import storage
import extraction

%matplotlib inline

In [2]:
log_file_path = "../"+settings.log_config
def setupLogging():
    with open(log_file_path, 'rt') as file:
        config = yaml.safe_load(file.read())
        logging.config.dictConfig(config)
setupLogging()

In [ ]:
setupLogging()

In [ ]:
# Read TSV files
train_rawdata_file_path = settings.train_raw_data
validation_rawdata_file_path = settings.validation_raw_data

#train_df = pd.read_table(train_rawdata_file_path, header = None, names = ['caption', 'url'] )
validate_df = pd.read_table(validation_rawdata_file_path, header = None, names = ['caption', 'url']) 

#train_df.head(20)
validate_df.head(5)

In [ ]:
train_df[train_df['caption'].str.contains("panda")]

In [ ]:
# Extracts image and caption from tsv and store padded(image),caption and HTTPStatus code
# into a train or validation hdf5 file

# Set image size
img_size = 300 , 300 
start_index = 0

# Extraction and Storage
#extraction.request_data_and_store(validate_df, size, "validate", start_index) 
#extraction.request_data_and_store(train_df, size, "train", start_index - 100)
extraction.request_data_and_store(validate_df, img_size, "train_le_retour", start_index)
 

In [3]:
def extract_data(set_name, img_size, start_index=0, extraction_size=100000):
# Extracts image and caption from tsv and store padded(image),caption and HTTPStatus code
# into a train or validation hdf5 file
    logger = logging.getLogger()
    data_df = None
    if set_name == "train":
        # Read TSV file
        train_rawdata_file_path = "../"+settings.train_raw_data
        data_df = pd.read_table(train_rawdata_file_path, header = None, names = ['caption', 'url'] )
        
    elif set_name == "validate":
        # Read TSV file
        validation_rawdata_file_path = "../"+settings.validation_raw_data
        data_df = pd.read_table(validation_rawdata_file_path, header = None, names = ['caption', 'url']) 

    else:
        logger.info("loader.extract_data: get yout set_name right! What is it you want to load again?")
        return
    
    # Extract images and stores images/captions
    #data_df = data_df.head(40)
    extraction.request_data_and_store(data_df, img_size, set_name, start_index, extraction_size)
    logger.info("loader.extract_data: all the data have been extracted and stored.")

In [7]:
# Get last stored index
#last_index = storage.get_last_stored_index("train")
#print(last_index)

size = 224, 224
extract_data("train", size, 1200005,10)

2020-10-03 16:51:05,820 - INFO <PID 46046:MainProcess> - MainThread - root - INFO - extraction:request_data_and_store-- Before ACQUIRE -- 1200010
2020-10-03 16:51:05,825 - INFO <PID 46046:MainProcess> - MainThread - root - INFO - extraction:request_data_and_store-- Before SUBMIT -- 1200010
2020-10-03 16:51:05,831 - INFO <PID 46046:MainProcess> - ThreadPoolExecutor-3_0 - root - INFO - train-- Fetching URL#1200010
2020-10-03 16:51:05,855 - INFO <PID 46046:MainProcess> - MainThread - root - INFO - extraction:request_data_and_store-- Before ACQUIRE -- 1200011
2020-10-03 16:51:05,859 - INFO <PID 46046:MainProcess> - MainThread - root - INFO - extraction:request_data_and_store-- Before SUBMIT -- 1200011
2020-10-03 16:51:05,862 - INFO <PID 46046:MainProcess> - ThreadPoolExecutor-3_1 - root - INFO - train-- Fetching URL#1200011
2020-10-03 16:51:05,877 - INFO <PID 46046:MainProcess> - MainThread - root - INFO - extraction:request_data_and_store-- Before ACQUIRE -- 1200012
2020-10-03 16:51:05,88

In [ ]:
from config import settings
print(settings.data_path)
print(settings.start_seq)
print(settings.end_seq)
print(settings.image_shape[2])


In [ ]:
request_data_and_store("xxtrain", 10, 20 )

In [ ]:
 extraction.extract_resnet_features_and_store("train", 10184, 100000)

In [ ]:
def clean_caption(caption):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)

    # tokenize
    desc = caption.split()
    # convert to lower case
    desc = [word.lower() for word in desc]
    # remove punctuation from each token
    desc = [w.translate(table) for w in desc]
    # remove hanging 's' and 'a'
    desc = [word for word in desc if len(word)>1]
    # remove tokens with numbers in them
    desc = [word for word in desc if word.isalpha()]
    # store as string
    caption = settings.start_seq +" "+ ' '.join(desc) +" "+ settings.end_seq
    return caption
            


In [ ]:
clean_caption("Les 3 petits cochons $ et *?")

In [ ]:
def clean_caption(caption):
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)

    # tokenize
    desc = caption.split()
    # convert to lower case
    desc = [word.lower() for word in desc]
    # remove punctuation from each token
    desc = [w.translate(table) for w in desc]
    # remove hanging 's' and 'a'
    desc = [word for word in desc if len(word)>1]
    # remove tokens with numbers in them
    desc = [word for word in desc if word.isalpha()]
    # store as string
    #caption = settings.start_seq +" "+ ' '.join(desc) +" "+ settings.end_seq
    return ' '.join(desc)
            

In [ ]:
def final_caption(caption):
    desc = settings.start_seq +" "+ clean_caption(caption) +" "+ settings.end_seq
    return desc

In [ ]:
final_caption("Les 3 petits cochons $ et *?")